## Data Initializing

In [1]:
import pandas as pd
import numpy as np
import data_handler as dh
import model as md
import time
import matplotlib.pyplot as plt
from matplotlib import rc
import pickle

In [2]:
import importlib
importlib.reload(dh)
importlib.reload(md)

<module 'model' from '/Users/kate/PycharmProjects/TransportNet-master/model.py'>

In [3]:
import numba
numba.__version__

'0.48.0'

# Stable Dynamics Model

parameter $\mu = 0$

In [4]:
sd_save = './'
net_name = 'Custom_net.tntp'
trips_name = 'Custom_trips.tntp'

In [5]:
handler = dh.DataHandler()
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)
graph_data = handler.GetGraphData(net_name, columns = ['init_node', 'term_node', 'capacity', 'free_flow_time'])
init_capacities = np.copy(graph_data['graph_table']['capacity'])

In [6]:
graph_data

{'nodes number': 2,
 'links number': 4,
 'zones number': 2,
 'graph_table':    init_node  init_node_thru  term_node  term_node_thru  capacity  \
 0          1            True          2            True    2000.0   
 1          1            True          2            True    2000.0   
 2          2            True          1            True    2000.0   
 3          2            True          1            True    2000.0   
 
    free_flow_time  
 0             1.0  
 1             0.5  
 2             1.0  
 3             0.5  }

In [7]:
graph_data['graph_table']['capacity'].head()

0    2000.0
1    2000.0
2    2000.0
3    2000.0
Name: capacity, dtype: float64

## Base flows
First of all, we should find admissible set of flows on the transport graph. It is required for defining duality gap.

In [8]:
alpha = 0.9 # 0.75
graph_data['graph_table']['capacity'] = init_capacities # * alpha
model = md.Model(graph_data, graph_correspondences,
                 total_od_flow, mu = 0)

graph_data['graph_table'].head()

[[2000.]
 [2000.]
 [2000.]
 [2000.]]


,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,True,2,True,2000.0,1.0
1,1,True,2,True,2000.0,0.5
2,2,True,1,True,2000.0,1.0
3,2,True,1,True,2000.0,0.5


In [9]:
import numba

In [10]:
numba.from_dtype(model.graph.in_edges(0).dtype)

int64

In [11]:
assert(model.mu == 0)
max_iter = 2000

solver_kwargs = {'eps_abs': 100, 'max_iter': max_iter, 'stop_crit': 'max_iter',
                 'verbose': True, 'verbose_step': 1000, 'save_history': True}
tic = time.time()
result = model.find_equilibrium(solver_name = 'ustm', composite = True,
                    solver_kwargs = solver_kwargs,
                    base_flows = alpha * graph_data['graph_table']['capacity'])
                    #base_flows here doesn't define anything now
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))
print('Time ratio =', 
      np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
print('Flow excess =', 
      np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

result['elapsed_time'] = toc - tic

base_flows = result['flows']

Composite optimization...
Oracles created...
Universal similar triangles method...
Primal_init = 3000
Dual_init = -3000
Duality_gap_init = 2000

Iterations number: 1000
Inner iterations number: 2011
Primal_func_value = 3975.97
Dual_func_value = -3999.97
Duality_gap = 128.762
Duality_gap / Duality_gap_init = 0.0643812


KeyboardInterrupt: 

In [ ]:
print(result['flows'])
# print(result['times'])

In [ ]:
#with open(sd_save + 'anaheim_' + 'ustm' + '_base_flows_max_iter_' + str(max_iter) + '_SD.pickle', 'wb') as f:
#    pickle.dump(base_flows, f)

## Model

In [ ]:
with open(sd_save + 'anaheim_' + 'ustm' + '_base_flows_max_iter_' + str(1000) + '_SD.pickle', 'rb') as f:
    base_flows = pickle.load(f)

In [ ]:
epsilons = np.logspace(2,0,9)
epsilons

In [ ]:
graph_data['graph_table']['capacity'] = init_capacities
model = md.Model(graph_data, graph_correspondences,
                 total_od_flow, mu = 0)

graph_data['graph_table'].head()

## Universal Similar Triangles

In [ ]:
assert(model.mu == 0)
max_iter = 40000
for index, eps_abs in enumerate(epsilons):
    print('eps_abs =', eps_abs)
    solver_kwargs = {'eps_abs': eps_abs,
                     'max_iter': max_iter, 'stop_crit': 'dual_gap',
                     'verbose': True, 'verbose_step': 2000, 'save_history': True}
    tic = time.time()
    result = model.find_equilibrium(solver_name = 'ustm', composite = True,
                                    solver_kwargs = solver_kwargs, base_flows = base_flows)
    toc = time.time()
    print('Elapsed time: {:.0f} sec'.format(toc - tic))
    print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
    print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

    result['eps_abs'] = eps_abs
    result['elapsed_time'] = toc - tic
    with open(sd_save + 'anaheim_result_' + 'ustm' + '_eps_abs_' + str(index) + '_SD.pickle', 'wb') as f:
        pickle.dump(result, f)